In [1]:
# anomalib_test

> Test anomalib functions

In [6]:
#| default_exp anomlib.train


In [7]:
#|  hide
%load_ext autoreload
%autoreload 2   

In [8]:
#| export
import pandas as pd
import numpy as np
from pathlib import Path
import os
from fastcore.all import *
from tqdm.auto import tqdm

In [9]:
# export
from cv_tools.core import *
from cv_tools.compress_and_filter import *
#from fastai.vision.all import PILImage

# Decmpress the parquet files

In [11]:
data_path = Path(os.getenv('DATA_PATH'), 'crack_detect_usm')
data_path.ls()

(#6) [Path('/home/user/Schreibtisch/projects/data/crack_detect_usm/good.parquet'),Path('/home/user/Schreibtisch/projects/data/crack_detect_usm/test_data_sig2.parquet'),Path('/home/user/Schreibtisch/projects/data/crack_detect_usm/test_data_sig4.parquet'),Path('/home/user/Schreibtisch/projects/data/crack_detect_usm/test_data_sig3.parquet'),Path('/home/user/Schreibtisch/projects/data/crack_detect_usm/test_data_sig1.parquet'),Path('/home/user/Schreibtisch/projects/data/crack_detect_usm/good_images')]

In [15]:
#pr_path = Path(data_path, 'good.parquet')
pr_path = Path(data_path, 'test_data_sig1.parquet')
decode_batch(
    pr_path=pr_path,
    output_dir=Path(data_path, 'test_data_sig1'),
    num_workers=4,
    batch_size=20
)

/home/user/miniconda3/envs/anomalib_dataset/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
#decode_batch(
    #pr_path=pr_path,
    #batch_size=600,
    #num_workers=4,
    #output_dir=Path(data_path, 'good_images'),
#)

In [1]:
#| export
from typing import Any

import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torchvision.transforms import ToPILImage

from anomalib import TaskType
from anomalib.data import MVTec
from anomalib.data.utils import read_image
from anomalib.deploy import OpenVINOInferencer, ExportType
from anomalib.engine import Engine
from anomalib.models import Padim,Patchcore

In [2]:
!pip list | grep anom

anomalib                  1.1.0
anomaly_detection         0.0.1          /home/user/Schreibtisch/projects/git_data/anomaly_detection


In [4]:

from anomalib.data.image.folder import Folder, FolderDataset
import numpy as np
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint

In [16]:
folder_datamodule = Folder(
    name="crack",
    root='/home/user/Schreibtisch/projects/data/crack_detect_usm',
    normal_dir="good_images",
    abnormal_dir="test_data_sig1",
    task=TaskType.CLASSIFICATION,
    image_size=(224, 224)
)
folder_datamodule.setup()
callbacks = [
    ModelCheckpoint(
        mode="max",
        monitor="image_AUROC",
    ),
    EarlyStopping(
        monitor="image_AUROC",
        mode="max",
        patience=3,

    ),
]
#engine = Engine(
    #normalization=NormalizationMethod.MIN_MAX,
    #threshold="F1AdaptiveThreshold",
    #task=TaskType.CLASSIFICATION,
    #image_metrics=["AUROC"],
    #accelerator="auto",
    #check_val_every_n_epoch=1,
    #devices=1,
    #max_epochs=1,
    #num_sanity_val_steps=0,
    #val_check_interval=1.0,
#)

engine = Engine(
    callbacks=callbacks,
    pixel_metrics="AUROC",
    accelerator="gpu",  # \<"cpu", "gpu", "tpu", "ipu", "hpu", "auto">,
    devices=0,
    logger=False,
    max_epochs=20,
    task=TaskType.CLASSIFICATION,
)

model = Padim(
    backbone='resnet18',
    layers=["layer1", "layer2", "layer3"],
    n_features=100
)
#model = Patchcore(
    #backbone='resnet18',
    #layers=["layer1", "layer2", "layer3"],
#)

engine.fit(
    datamodule=folder_datamodule, 
    model=model,
    )

Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/user/miniconda3/envs/anomalib_dataset/lib/python3.11/site-packages/lightning/pytorch/core/optimizer.py:181: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer


┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name                  ┃ Type                     ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model                 │ PadimModel               │  2.8 M │
│ 1 │ _transform            │ Compose                  │      0 │
│ 2 │ normalization_metrics │ MinMax                   │      0 │
│ 3 │ image_threshold       │ F1AdaptiveThreshold      │      0 │
│ 4 │ pixel_threshold       │ F1AdaptiveThreshold      │      0 │
│ 5 │ image_metrics         │ AnomalibMetricCollection │      0 │
│ 6 │ pixel_metrics         │ AnomalibMetricCollection │      0 │
└───┴───────────────────────┴──────────────────────────┴────────┘

Trainable params: 2.8 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 2.8 M                                                                                                
Total estimated model params size (MB): 11

Output()

/home/user/miniconda3/envs/anomalib_dataset/lib/python3.11/site-packages/lightning/pytorch/loops/optimization/autom
atic.py:132: `training_step` returned `None`. If this was on purpose, ignore this warning...

In [ ]:
test_results = engine.test(model=model, datamodule=folder_datamodule)

In [ ]:
openvino_model_path = engine.export(
    model=model,
    export_type=ExportType.OPENVINO,
    export_root=str(Path.cwd()),
)